In [1]:
import tensorflow as tf
import numpy as np

# np로 데이터 읽어 들이고 변환
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [2]:
# 모델 저장 할때 쓸 변수 생성, 학습 사용x, 학습 횟수 카운트, trainable= False
global_step = tf.Variable(0, trainable = False, name='global_step')

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망을 하나 더 늘리고 대신 가중치만 넣고 편향을 빼보자 구성이 명확해진다.
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
# global_step을 넘겨 줬다. 이렇게 하면 최적화 함수가 변수 최적화 할때마다 변수값 1씩 증가
train_op = optimizer.minimize(cost, global_step=global_step)


In [4]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables()) # global_variables()는 앞서 정의한 변수들을 가져오는 함수

In [13]:
# ./model 디렉토리에 기존 학습 모델 있는지 확인해서 있다면 saver.restore로 가져오고 아니면 초기화
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [14]:
# 가져온 것이니 2번만 재학습
for step in range(2):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X:x_data, Y:y_data}))

Step: 53,  Cost: 0.555
Step: 54,  Cost: 0.555


In [15]:
# 지정한 파일에 저장
saver.save(sess, './model/dnn.ckpt', global_step)

'./model/dnn.ckpt-54'

In [16]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y:y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
